In [1]:
import re
import PyPDF2
import string
import csv
import collections

In [2]:
Megadb = {}
# Megadb[0][Brian][1][0]
Megadb[0]= {}
Megadb[0]["Brian"]=(0,[])
Megadb[0]["Brian"][1].append("Hi I'm Brian")

print(Megadb[0]["Brian"][1][0])

#Make a copy
MegadbClean = Megadb

Hi I'm Brian


In [3]:
def get_confessionals (pdf_name, csv_name):
    pdfFileObj = open(pdf_name, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    all_text = ""
    for i in range(0,pdfReader.numPages):
        pageObj = pdfReader.getPage(i)
        all_text+=pageObj.extractText().replace("\n","").replace("\'","'").replace("(laughs)","").replace("!",".").replace("\"","").replace("?",".").replace("...","").replace(",","")
    iter = re.finditer(r"\d\.", all_text)
    indices = [m.start(0) for m in iter]
    
    for z in range (2):
        j=0
        hold_string = ""
        names = ""
        for i in range (0, len(indices)):
            if i == len(indices):
                break
            while all_text[indices[i]+j] is not "(":
                hold_string+=all_text[indices[i]+j]
                j+=1
                if j > 30:
                    del indices[i]
                    i = i-1
                    hold_string=""
                    break
            '''if j < 4:
                del indices[i]
                i = i-1
                hold_string=""'''
            names+=hold_string
            hold_string=""
            j=0
        names = ''.join(i for i in names if not i.isdigit())
        count = 0
        names = ''.join(names.split())
        names = names[1:]
    
    flawed_name_list = []
    hold = ""
    for n in range(0, len(names)):
        if names[n] is not ".":
            hold+=names[n]
        if names[n] == '.' or n == len(names)-1:
            flawed_name_list.append(hold)
            hold=""
    rank = []
    all_names = []
    i=1
    s=""
    with open(csv_name) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        for row in csvReader:
            rank.append(i)
            s = ''.join(row[1].split())
            all_names.append(s)
            i+=1
    name_list = []
    good_nums = []
    for x in range(0,len(flawed_name_list)):
        for y in range (0, len(all_names)):
            if flawed_name_list[x] == all_names[y]:
                name_list.append(flawed_name_list[x])
                good_nums.append(x)
    sequential_list = [i for i in range (0, len(flawed_name_list))]
    bad = []
    bad = list(set(sequential_list) - set(good_nums))
    start = 0
    clear = False
    hold = ""
    confessionals = []
    for i in range (0, len(all_text)):
        if i in indices:
            clear = False
        if clear is False:
            confessionals.append(hold)
            hold = ""
            clear= True
        if clear:
            hold+=all_text[i]
    confessionals = confessionals[1:]
    delete=0
    cleaned_confessionals = []
    for confessional in confessionals:
        delete = re.search(':',confessional).start()
        cleaned_confessionals.append(confessional[delete+2:-1])
    season={}
    for j in range (0, len(all_names)):
        hold = []
        for i in range (0, len(cleaned_confessionals)):
            if name_list[i] == all_names[j]:
                hold.append(cleaned_confessionals[i])
        season[all_names[j]] = (rank[j], hold)
    return(season)

In [4]:
Megadb[1] = get_confessionals('season2new.pdf', 'season2_rank.csv')
#print (Megadb[1])
print (Megadb[1]["Tina"][1])

["You know we're not getting eight hours' sleep. We're not getting you know but a handful of food a day. But you know this is the game we signed up to play. ", "It's pasty. It tastes like glue. It just in your mouth it goes (smacking lips) It's just horrible and I can't eat it. ", "Jerri's gonna take that job over which I hope doesn't hurt Keith's feelings. You know I mean that's his area and it's kind of a slap in the face. ", "You know it was like a big feeding frenzy. You know Kel's got beef jerky and they said Look in his backpack real quick and I was standing right there and I did it. And there wasn't anything in there at all. But still and yet all I was thinking about If I find beef jerky in here it's gonna be ugly. ", "I'll definitely feel bad about having to vote somebody off. There's some humiliation involved with being the first one voted off. ", "To be honest I didn't expect this adventure to be this hard. I find myself just craving food. Doritos-- I want a bag of Doritos mo

In [5]:
# Method to give a list of all the words in string
def just_words(input):
	letters_only = re.sub("[^a-zA-Z]",   # The pattern to search for
                      " ",               # The pattern to replace it with
                      input)             # The text to search
	lower_case = letters_only.lower()    # Convert to lower case
	words = lower_case.split()           # Split into words
	return words

In [6]:
# Get stopwords
import nltk
from nltk.corpus import stopwords # Import the stop word list
print(str(stopwords.words("english")))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [7]:
#Replace confessional lists with list of keywords for that person
test = [1]
for s in test:#Megadb.keys():
    for p in Megadb[s].keys(): 
        MegadbClean[s][p]= (Megadb[s][p][0],collections.Counter([w for w in just_words(str([i for i in Megadb[s][p][1]])) if w not in set(stopwords.words("english"))]))
print (MegadbClean[1])

["You know we're not getting eight hours' sleep. We're not getting you know but a handful of food a day. But you know this is the game we signed up to play. ", "It's pasty. It tastes like glue. It just in your mouth it goes (smacking lips) It's just horrible and I can't eat it. ", "Jerri's gonna take that job over which I hope doesn't hurt Keith's feelings. You know I mean that's his area and it's kind of a slap in the face. ", "You know it was like a big feeding frenzy. You know Kel's got beef jerky and they said Look in his backpack real quick and I was standing right there and I did it. And there wasn't anything in there at all. But still and yet all I was thinking about If I find beef jerky in here it's gonna be ugly. ", "I'll definitely feel bad about having to vote somebody off. There's some humiliation involved with being the first one voted off. ", "To be honest I didn't expect this adventure to be this hard. I find myself just craving food. Doritos-- I want a bag of Doritos mo

['That trek let me know that this was for real. That was a long trek we had through deep sand with a lot of heavy equipment and I was packing more weight than anyone else so I was really struggling. It was a long trip and that was that fired me up because that is the quickest way to find out what people are made of.', "I really did bring the flag to serve as shelter but you know don't get me wrong. When I wake up in the morning there's two things I'm thankful for. I'm thankful I'm alive and I'm thankful I'm a Texan. ", "Keith is just licking his chops. He's wanting to cook so bad he can't stand it and so you can bet he's going to work on this fire thing until we lick it. ", "Kel's done a little bit of fishing which just I mean come on the guy couldn't fish a rubber ducky out of a bathtub. The bait choices the-the lines He's just going about it the wrong way. ", "If you were looking to paint a bull's-eye on someone obviously Keith has struck a nerve in a few people. Um but also Jerri. S

["We don't know how far away we are so I'm going to run up this and do a little reconnaissance see if we can find out where we're at. I figure if it's really far we'll drape some of our stuff here and we won't take it. If we're not far then we'll buck up and get it there.", 'As soon as we hit the beach you know it was really kind of amazing. Collectively we all dispersed to do something different. ', "I've never started a fire before from scratch but you know I knew nightfall was coming. We'd have no light and that we'd be able to cook nothing um and we wouldn't have a bonfire so it was it was pretty critical. ", 'Since Jerri and I had our little confrontation before I thought you know this is a great time to get her involved with this process to be a part of it somehow with something she brought that was important to her. ', "Our plan for getting food is uh We don't really have a good plan actually. ", "So there's so much work to do as a team that uh and it's hard physical labor to co

["I mean everything was heavy to carry. There's you can't discount any of our equipment as easy to carry but some of it was just more awkward. And some of the crossings we had to go you know down a sharp valley of rocks and up again and kind of wind and balance across a river. And it was frustrating too but with the water jugs and you know the water canteens and I mean that stuff is just awkward. ", "As soon as we got out of the river I think we just this kind of realization. We got a little cold and we realized that we're going to We have to sleep somewhere and we just wanted to feel a little safe and dry. So we got to work as much as we could on building the basis of a home. ", "Winning immunity is the most important thing. We need to beat beat beat the other tribe. We'll let them eliminate someone on their team first. ", "We have one knife and that one knife it might as well be food but he wants to go and spear a pig. Really. Have we seen a pig. We haven't seen a pig. I haven't seen

["One thing we tried to do in starting the fire was use my Bible which I had brought along. It was my luxury item. And uh we tore some pages out of it. Uh it wasn't the pages that actually had any scripture in it but I would've used that. You know if you're you got to do with what you got. ", 'That was a nice little piece of home when you when you least least expect it you know. Then it said go to page 200 and then go to page 150 and we went through that about seven or eight times I think till we till we finally came to the end. And it was a little sentimental for me to tell you the truth to hear it. ', "Yesterday's Immunity Challenge was a big letdown. Uh uh not only physically but emotionally. I think probably more so emotionally. As far as the Tribal Council goes tonight which are which we have to go to uh I don't think any of us are looking forward to it. ", "Today's challenge is I believe is probably going to be jumping off a cliff into some water. Water is not my finest my uh fin

["(groans) I'm a little tired. This rice is very heavy. I really want to get there soon.", "Keith says he's a chef but and it's not that I don't believe that he can't cook very well you know. And it's not I'm not doubting his talent. But I think it might be that he's adding maybe too much water to the rice. ", "I'm worried about running out of food and running out of rice. And we've been trying to eat it sparingly but It's going to run out sooner or later and uh I would love to catch a fish out there or something. ", "Today is Day 7 and we have caught our first fish so actually that's not too bad. It did take us a week but hey it's better than two weeks. ", "There could be something going on between Jerri and Colby. I mean from the very first day she told me how cute she thought he was. And at first she didn't think that he liked her. And I was like Jerri it's only the second day. ", "It's a concern um but you know I think we'll be safe as long as we head for the water and everybody do

["Keith actually ran ahead to go scout out where the camp was 'cause we were all getting pretty tired and the last leg of that walk was really difficult.", 'When we got to our camp I was considering at that point anything and everything anybody was doing or saying as a strategic move. ', 'The tension between Keith and I I think him and I kind of rubbed each other the wrong way. ', "Keith and I didn't start off on a very good foot because of some of the condescending mark-remarks he made to me and he might very well not even be aware that he does it but I'm not the kind of person to just let it slide off my back. ", "Believe me when he goes out I'm hoping more than anybody that he's going to come back with some fish and some shrimp and we will have a big seafood buffet. But it's not happening. And it's  He's just he's not a fisherman obviously. He should keep his day job.  ", "It just it didn't make sense. It didn't make sense. Like how could a gourmet chef not know how to make rice. I 

['When they dropped us off you know everybody was designated a certain thing to do and uh they gave Nick the compass. For some reason although the compass said one way we kept going the wrong way.', "Mike was hungry. That's why he did that. And-and that is another indication of Mike being Mike; about himself.", "He really thinks he is leader of this tribe. I'm telling you now that there are people in this tribe that want to vote him off already. ", "Kimmi's not going to compromise her morals. But coming into a game like this you can't come in with a whole list of things you're not going to do and you're not going to eat because you're not helping your tribe you're not helping your team morale. ", 'Every job and task that Kimmi takes on in this camp can be taken over by somebody else for sure. ', "Kentucky Joe although he's not a strong member of this tribe as far as challenges are concerned he knows what he's doing outside. ", "Elisabeth said she won't vote off Kentucky Joe and-and she

['When we started building the shelter I thought everybody started chipping in because there was the wood gatherers and there was the people that were building and uh I had worked a lot of uh you know some construction in my life and I-I had a reasonable sense of uh of-of construction. ', "I think we're starting to get to know each other and people are not accustomed to this kind of lifestyle yet. I'm a little accustomed to this lifestyle because I do hunt and I do fish and I do camp so it's not that foreign to me. When I saw those figs it was a big rush. And two or three people came over and helped pick the figs and we brought 'em back and uh I bit into one right off the tree. ", "The people that I think that are in jeopardy tonight are potentially Jeff because of the health issues and Rodger might not be safe. He could be a liability in the challenges and especially swimming challenges and climbing challenges. And it's not just Rodger. It would be anybody over 50 years old. ", "Every

['Colby-- his luxury item was the Texan flag which was great because it served as the tarp that covers covers us and blocks us from the wind and the sun. ', "This Immunity Challenge is extremely important because we have to show the other team that we're better you know and plus we have to have fire in order to start eating rice. ", "Kel is like the fisherman of the group. He's very persistent in that area and part of me thinks it's because he feels if he does catch a fish that might um keep him here longer.", "I say stop wasting the energy on fishing. And I don't think there's any fish out there. ", 'Jerri blatantly said that she saw something that looked just like beef jerky go into his mouth and I I believe her actually. ', "Keith is um he's a bit bossy. He's very condescending and he's definitely close to going. ", "I think the name of the game is trust no one. I don't think you could trust your own grandparents out here. I mean I think I would vote them right off just 'cause  they

In [8]:
# Method taking two input lists giving a dictionary of the first list and a 1 or 0 depending on if the word appears in the second list
def match(list, comp_list):
    D = {}
    for w in trim:
        if w in comp_words:
            D[w]=1
        else:
            D[w]=0

    return(D)

# Dictionary with input compared to comparison list with 1's and 0's
comp = match(trim, comp_words)
print(comp)

NameError: name 'trim' is not defined

In [ ]:
# Print just the 1's and 0's
print(list(comp.values()))

In [ ]:
# Counter of similarites 
counter=collections.Counter(list(comp.values()))
print(counter.values())
sim = float(counter[1]/(counter[0]+counter[1]))   #percent of words that appeared in the compared text
print("Percent Similarity: " + str(sim))